# EBAC - Regressão II - regressão múltipla

## Tarefa I

# EBAC - Regression II - Multiple Regression
## Task I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

#### Income Prediction

We will be working with the 'income_prediction.csv' dataset, which is the basis for your next project. We'll utilize the resources we've covered so far in this dataset.

|variable|description|
|-|-|
|data_ref                | Reference date for the collection of variables|
|index                   | Customer identification code|
|sexo                    | Customer's gender|
|posse_de_veiculo        | Indicates if the customer owns a car|
|posse_de_imovel         | Indicates if the customer owns a property|
|qtd_filhos              | Number of children the customer has|
|tipo_renda              | Customer's income type|
|educacao                | Customer's level of education|
|estado_civil            | Customer's marital status|
|tipo_residencia         | Customer's type of residence (owned, rented, etc)|
|idade                   | Customer's age|
|tempo_emprego           | Duration in the current job|
|qt_pessoas_residencia   | Number of people living in the residence|
|renda                   | Income in Brazilian Real (BRL)|

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for maior que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
<br>

1. Fit a model to predict log(income) considering all available covariates.
   - Using Patsy resources, transform qualitative variables into dummies.
   - Always keep the most frequent category as the reference cell.
   - Evaluate the parameters and see if they seem practically sensible.

2. Remove the least significant variable and analyze:
   - Observe the indicators we've covered and assess whether, in your opinion, the model improved or worsened.
   - Examine the parameters and check if any have changed significantly.
   
3. Continue removing the least significant variables whenever the p-value is greater than 5%. Compare the final model with the initial one. Observe the indicators and conclude whether the model appears better.    

In [246]:
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [247]:
df = pd.read_csv('previsao_de_renda.csv')
drop = df[['Unnamed: 0','id_cliente','data_ref']]
df = df.drop(drop, axis = 1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sexo                   15000 non-null  object 
 1   posse_de_veiculo       15000 non-null  bool   
 2   posse_de_imovel        15000 non-null  bool   
 3   qtd_filhos             15000 non-null  int64  
 4   tipo_renda             15000 non-null  object 
 5   educacao               15000 non-null  object 
 6   estado_civil           15000 non-null  object 
 7   tipo_residencia        15000 non-null  object 
 8   idade                  15000 non-null  int64  
 9   tempo_emprego          12427 non-null  float64
 10  qt_pessoas_residencia  15000 non-null  float64
 11  renda                  15000 non-null  float64
dtypes: bool(2), float64(3), int64(2), object(5)
memory usage: 1.2+ MB


In [248]:
df.isnull().sum()

sexo                        0
posse_de_veiculo            0
posse_de_imovel             0
qtd_filhos                  0
tipo_renda                  0
educacao                    0
estado_civil                0
tipo_residencia             0
idade                       0
tempo_emprego            2573
qt_pessoas_residencia       0
renda                       0
dtype: int64

In [249]:
df = df.dropna()
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12427 entries, 0 to 12426
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sexo                   12427 non-null  object 
 1   posse_de_veiculo       12427 non-null  bool   
 2   posse_de_imovel        12427 non-null  bool   
 3   qtd_filhos             12427 non-null  int64  
 4   tipo_renda             12427 non-null  object 
 5   educacao               12427 non-null  object 
 6   estado_civil           12427 non-null  object 
 7   tipo_residencia        12427 non-null  object 
 8   idade                  12427 non-null  int64  
 9   tempo_emprego          12427 non-null  float64
 10  qt_pessoas_residencia  12427 non-null  float64
 11  renda                  12427 non-null  float64
dtypes: bool(2), float64(3), int64(2), object(5)
memory usage: 995.3+ KB


<br><br>
1. Fit a model to predict log(income) considering all available covariates.
   - Using Patsy's resources, encode qualitative variables as dummies.
   - Always keep the most frequent category as the reference cell.
   - Evaluate the parameters and see if they seem to make practical sense.

In [250]:
y, x = patsy.dmatrices('''np.log(renda) ~ C(sexo, Treatment(0)) + C(posse_de_veiculo, Treatment(0)) 
                       + C(posse_de_imovel, Treatment(1)) + qtd_filhos 
                       + C(tipo_renda, Treatment(0)) + C(educacao, Treatment(2))
                       + C(estado_civil, Treatment(0)) + C(tipo_residencia, Treatment(1))
                       + idade + tempo_emprego + qt_pessoas_residencia''', data=df)

reg = sm.OLS(y, x).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Tue, 07 Nov 2023   Prob (F-statistic):               0.00
Time:                        19:33:06   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
=====================================================================================================================
                                                        coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------
Intercept                                             6.6093      0.219     30.214      0.000       6.181       7.038
C(sexo, Treatment(0))[T.M]                            0.7874      0.015     53.723      0.000       0.759       0.816
C(posse_de_veiculo, Treatment(0))[T.True]             0.0441      0.014      3.119      0.002       0.016       0.072
C(posse_de_imovel, Treatment(1))[T.False]            -0.0829      0.014     -5.926      0.000      -0.110      -0.055
C(tipo_renda, Treatment(0))[T.Bolsista]               0.2209      0.241      0.916      0.360      -0.252       0.694
C(tipo_renda, Treatment(0))[T.Empresário]             0.1551      0.015     10.387      0.000       0.126       0.184
C(tipo_renda, Treatment(0))[T.Pensionista]           -0.3087      0.241     -1.280      0.201      -0.782       0.164
C(tipo_renda, Treatment(0))[T.Servidor público]       0.0576      0.022      2.591      0.010       0.014       0.101
C(educacao, Treatment(2))[T.Primário]                 0.0141      0.072      0.196      0.844      -0.127       0.155
C(educacao, Treatment(2))[T.Pós graduação]            0.1212      0.142      0.853      0.394      -0.157       0.400
C(educacao, Treatment(2))[T.Superior completo]        0.1079      0.014      7.763      0.000       0.081       0.135
C(educacao, Treatment(2))[T.Superior incompleto]     -0.0290      0.032     -0.900      0.368      -0.092       0.034
C(estado_civil, Treatment(0))[T.Separado]             0.3265      0.111      2.935      0.003       0.108       0.545
C(estado_civil, Treatment(0))[T.Solteiro]             0.2672      0.109      2.453      0.014       0.054       0.481
C(estado_civil, Treatment(0))[T.União]               -0.0343      0.025     -1.361      0.174      -0.084       0.015
C(estado_civil, Treatment(0))[T.Viúvo]                0.3689      0.116      3.191      0.001       0.142       0.596
C(tipo_residencia, Treatment(1))[T.Aluguel]           0.0442      0.054      0.815      0.415      -0.062       0.150
C(tipo_residencia, Treatment(1))[T.Com os pais]       0.0184      0.029      0.625      0.532      -0.039       0.076
C(tipo_residencia, Treatment(1))[T.Comunitário]      -0.0754      0.091     -0.829      0.407      -0.254       0.103
C(tipo_residencia, Treatment(1))[T.Estúdio]           0.1107      0.084      1.319      0.187      -0.054       0.275
C(tipo_residencia, Treatment(1))[T.Governamental]    -0.0129      0.039     -0.331      0.741      -0.089       0.064
qtd_filhos                                           -0.2658      0.108     -2.463      0.014      -0.477      -0.054
idade                                                 0.0053      0.001      6.624      0.000       0.004       0.007
tempo_emprego  

In [251]:
# R²:
df['y_chap'] = np.exp(reg.fittedvalues)
df[['y_chap', 'renda']].corr().iloc[0,1]**2

0.3589920594855584

 - O uso de dummies é valioso porque permite que o modelo capture a variação associada a cada categoria em relação à referência. Sem a criação de dummies, o modelo poderia interpretar erroneamente a variável categórica. Faz sentido a utilização de Dummies porém é necessário lapidar os dados para que sejam relevantes.<br>
 - The use of dummy variables is valuable because it enables the model to capture the variation associated with each category concerning the reference. Without the creation of dummy variables, the model could misinterpret the categorical variable. It makes sense to use dummy variables, but it's necessary to refine the data to make them relevant.

<br><br>
2. Remove the least significant variable and analyze:

   - Observe the indicators we've seen and evaluate if the model improved or worsened in your opinion.
   - Examine the parameters and see if any have changed significantly.

In [252]:
y, x = patsy.dmatrices('''np.log(renda) ~ C(sexo, Treatment(0)) + C(posse_de_veiculo, Treatment(0)) 
                       + C(posse_de_imovel, Treatment(1)) + qtd_filhos 
                       + C(tipo_renda, Treatment(0)) + C(estado_civil, Treatment(0)) 
                       + C(tipo_residencia, Treatment(1)) + idade + tempo_emprego 
                       + qt_pessoas_residencia''', data=df)

sm.OLS(y, x).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.354
Model:                            OLS   Adj. R-squared:                  0.353
Method:                 Least Squares   F-statistic:                     340.0
Date:                Tue, 07 Nov 2023   Prob (F-statistic):               0.00
Time:                        19:33:07   Log-Likelihood:                -13601.
No. Observations:               12427   AIC:                         2.724e+04
Df Residuals:                   12406   BIC:                         2.740e+04
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
=====================================================================================================================
                                                        coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------
Intercept                                             6.6755      0.219     30.481      0.000       6.246       7.105
C(sexo, Treatment(0))[T.M]                            0.7800      0.015     53.208      0.000       0.751       0.809
C(posse_de_veiculo, Treatment(0))[T.True]             0.0533      0.014      3.769      0.000       0.026       0.081
C(posse_de_imovel, Treatment(1))[T.False]            -0.0856      0.014     -6.110      0.000      -0.113      -0.058
C(tipo_renda, Treatment(0))[T.Bolsista]               0.2988      0.242      1.237      0.216      -0.175       0.772
C(tipo_renda, Treatment(0))[T.Empresário]             0.1651      0.015     11.079      0.000       0.136       0.194
C(tipo_renda, Treatment(0))[T.Pensionista]           -0.2513      0.242     -1.039      0.299      -0.725       0.223
C(tipo_renda, Treatment(0))[T.Servidor público]       0.0754      0.022      3.400      0.001       0.032       0.119
C(estado_civil, Treatment(0))[T.Separado]             0.3238      0.112      2.903      0.004       0.105       0.542
C(estado_civil, Treatment(0))[T.Solteiro]             0.2616      0.109      2.396      0.017       0.048       0.476
C(estado_civil, Treatment(0))[T.União]               -0.0374      0.025     -1.481      0.139      -0.087       0.012
C(estado_civil, Treatment(0))[T.Viúvo]                0.3601      0.116      3.107      0.002       0.133       0.587
C(tipo_residencia, Treatment(1))[T.Aluguel]           0.0369      0.054      0.680      0.496      -0.069       0.143
C(tipo_residencia, Treatment(1))[T.Com os pais]       0.0185      0.029      0.627      0.530      -0.039       0.076
C(tipo_residencia, Treatment(1))[T.Comunitário]      -0.0403      0.091     -0.443      0.658      -0.218       0.138
C(tipo_residencia, Treatment(1))[T.Estúdio]           0.1225      0.084      1.456      0.145      -0.042       0.287
C(tipo_residencia, Treatment(1))[T.Governamental]    -0.0209      0.039     -0.535      0.593      -0.098       0.056
qtd_filhos                                           -0.2567      0.108     -2.372      0.018      -0.469      -0.045
idade                                                 0.0049      0.001      6.185      0.000       0.003       0.006
tempo_emprego                                         0.0617      0.001     59.327      0.000       0.060       0.064
qt_pessoas_residencia                                 0.2846      0.108      2.634      0.008       0.073       0.496
==============================================================================
Omnibus:                        1.217   Durbin-Watson:                   2.024
Prob(Omnibus):                  0.544   Jarque-Bera (JB):                1.187
Skew:         

 - Verificamos que o modelo começa a ter alterações de valores que se distanciam e outros que tornam mais relevantes e necessários para que sejam mantidos. Nenhum deles se alteraram de forma agressiva.<br>
 - "We observed that the model begins to show changes in values that distance themselves and others that become more relevant and necessary to be retained. None of them changed aggressively."
 <br><br><br>

3. Continue removing the less significant variables whenever the p-value is greater than 5%. Compare the final model with the initial one. Observe the indicators and conclude whether the model seems better."

In [253]:
# tipo_residencia
y, x = patsy.dmatrices('''np.log(renda) ~ C(sexo, Treatment(0)) + C(posse_de_veiculo, Treatment(0)) 
                       + C(posse_de_imovel, Treatment(1)) + qtd_filhos 
                       + C(tipo_renda, Treatment(0)) + C(estado_civil, Treatment(0)) 
                       + idade + tempo_emprego + qt_pessoas_residencia''', data=df)

sm.OLS(y, x).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.354
Model:                            OLS   Adj. R-squared:                  0.353
Method:                 Least Squares   F-statistic:                     453.1
Date:                Tue, 07 Nov 2023   Prob (F-statistic):               0.00
Time:                        19:33:07   Log-Likelihood:                -13603.
No. Observations:               12427   AIC:                         2.724e+04
Df Residuals:                   12411   BIC:                         2.736e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
===================================================================================================================
                                                      coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------
Intercept                                           6.6799      0.219     30.517      0.000       6.251       7.109
C(sexo, Treatment(0))[T.M]                          0.7819      0.015     53.480      0.000       0.753       0.811
C(posse_de_veiculo, Treatment(0))[T.True]           0.0535      0.014      3.789      0.000       0.026       0.081
C(posse_de_imovel, Treatment(1))[T.False]          -0.0848      0.014     -6.172      0.000      -0.112      -0.058
C(tipo_renda, Treatment(0))[T.Bolsista]             0.2998      0.242      1.241      0.215      -0.174       0.773
C(tipo_renda, Treatment(0))[T.Empresário]           0.1655      0.015     11.120      0.000       0.136       0.195
C(tipo_renda, Treatment(0))[T.Pensionista]         -0.2540      0.242     -1.051      0.293      -0.728       0.220
C(tipo_renda, Treatment(0))[T.Servidor público]     0.0769      0.022      3.472      0.001       0.033       0.120
C(estado_civil, Treatment(0))[T.Separado]           0.3241      0.112      2.907      0.004       0.106       0.543
C(estado_civil, Treatment(0))[T.Solteiro]           0.2623      0.109      2.403      0.016       0.048       0.476
C(estado_civil, Treatment(0))[T.União]             -0.0361      0.025     -1.431      0.153      -0.086       0.013
C(estado_civil, Treatment(0))[T.Viúvo]              0.3588      0.116      3.097      0.002       0.132       0.586
qtd_filhos                                         -0.2560      0.108     -2.366      0.018      -0.468      -0.044
idade                                               0.0048      0.001      6.147      0.000       0.003       0.006
tempo_emprego                                       0.0617      0.001     59.407      0.000       0.060       0.064
qt_pessoas_residencia                               0.2839      0.108      2.629      0.009       0.072       0.496
==============================================================================
Omnibus:                        1.149   Durbin-Watson:                   2.024
Prob(Omnibus):                  0.563   Jarque-Bera (JB):                1.121
Skew:                           0.021   Prob(JB):                        0.571
Kurtosis:                       3.019   Cond. No.                     2.13e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.13e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [254]:
#tipo_renda
y, x = patsy.dmatrices('''np.log(renda) ~ C(sexo, Treatment(0)) + C(posse_de_veiculo, Treatment(0)) 
                       + C(posse_de_imovel, Treatment(1)) + qtd_filhos 
                       + C(estado_civil, Treatment(0)) + idade + tempo_emprego + qt_pessoas_residencia''', data=df)

sm.OLS(y, x).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.347
Model:                            OLS   Adj. R-squared:                  0.347
Method:                 Least Squares   F-statistic:                     600.2
Date:                Tue, 07 Nov 2023   Prob (F-statistic):               0.00
Time:                        19:33:07   Log-Likelihood:                -13666.
No. Observations:               12427   AIC:                         2.736e+04
Df Residuals:                   12415   BIC:                         2.745e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
=============================================================================================================
                                                coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
Intercept                                     6.7873      0.220     30.885      0.000       6.357       7.218
C(sexo, Treatment(0))[T.M]                    0.7715      0.015     52.631      0.000       0.743       0.800
C(posse_de_veiculo, Treatment(0))[T.True]     0.0565      0.014      3.985      0.000       0.029       0.084
C(posse_de_imovel, Treatment(1))[T.False]    -0.0868      0.014     -6.288      0.000      -0.114      -0.060
C(estado_civil, Treatment(0))[T.Separado]     0.3096      0.112      2.763      0.006       0.090       0.529
C(estado_civil, Treatment(0))[T.Solteiro]     0.2533      0.110      2.309      0.021       0.038       0.468
C(estado_civil, Treatment(0))[T.União]       -0.0281      0.025     -1.110      0.267      -0.078       0.022
C(estado_civil, Treatment(0))[T.Viúvo]        0.3317      0.116      2.850      0.004       0.104       0.560
qtd_filhos                                   -0.2443      0.109     -2.247      0.025      -0.457      -0.031
idade                                         0.0046      0.001      5.783      0.000       0.003       0.006
tempo_emprego                                 0.0611      0.001     59.136      0.000       0.059       0.063
qt_pessoas_residencia                         0.2677      0.109      2.466      0.014       0.055       0.480
==============================================================================
Omnibus:                        1.331   Durbin-Watson:                   2.025
Prob(Omnibus):                  0.514   Jarque-Bera (JB):                1.311
Skew:                           0.024   Prob(JB):                        0.519
Kurtosis:                       3.013   Cond. No.                     2.13e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.13e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [255]:
# estado_civil
y, x = patsy.dmatrices('''np.log(renda) ~ C(sexo, Treatment(0)) + C(posse_de_veiculo, Treatment(0)) 
                       + C(posse_de_imovel, Treatment(1)) + qtd_filhos 
                       + idade + tempo_emprego + qt_pessoas_residencia''', data=df)

sm.OLS(y, x).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.347
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     940.8
Date:                Tue, 07 Nov 2023   Prob (F-statistic):               0.00
Time:                        19:33:07   Log-Likelihood:                -13672.
No. Observations:               12427   AIC:                         2.736e+04
Df Residuals:                   12419   BIC:                         2.742e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=============================================================================================================
                                                coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
Intercept                                     7.3243      0.044    167.889      0.000       7.239       7.410
C(sexo, Treatment(0))[T.M]                    0.7694      0.015     52.676      0.000       0.741       0.798
C(posse_de_veiculo, Treatment(0))[T.True]     0.0569      0.014      4.022      0.000       0.029       0.085
C(posse_de_imovel, Treatment(1))[T.False]    -0.0866      0.014     -6.275      0.000      -0.114      -0.060
qtd_filhos                                    0.0338      0.019      1.735      0.083      -0.004       0.072
idade                                         0.0049      0.001      6.408      0.000       0.003       0.006
tempo_emprego                                 0.0610      0.001     59.075      0.000       0.059       0.063
qt_pessoas_residencia                        -0.0092      0.016     -0.566      0.572      -0.041       0.023
==============================================================================
Omnibus:                        1.240   Durbin-Watson:                   2.025
Prob(Omnibus):                  0.538   Jarque-Bera (JB):                1.213
Skew:                           0.022   Prob(JB):                        0.545
Kurtosis:                       3.019   Cond. No.                         301.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [256]:
# qt_pessoas_residencia
y, x = patsy.dmatrices('''np.log(renda) ~ C(sexo, Treatment(0)) + C(posse_de_veiculo, Treatment(0)) 
                       + C(posse_de_imovel, Treatment(1)) + qtd_filhos 
                       + idade + tempo_emprego''', data=df)

reg = sm.OLS(y, x).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.347
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     1098.
Date:                Tue, 07 Nov 2023   Prob (F-statistic):               0.00
Time:                        19:33:08   Log-Likelihood:                -13673.
No. Observations:               12427   AIC:                         2.736e+04
Df Residuals:                   12420   BIC:                         2.741e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
=============================================================================================================
                                                coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
Intercept                                     7.3089      0.034    213.746      0.000       7.242       7.376
C(sexo, Treatment(0))[T.M]                    0.7688      0.015     52.768      0.000       0.740       0.797
C(posse_de_veiculo, Treatment(0))[T.True]     0.0561      0.014      3.986      0.000       0.029       0.084
C(posse_de_imovel, Treatment(1))[T.False]    -0.0866      0.014     -6.278      0.000      -0.114      -0.060
qtd_filhos                                    0.0239      0.009      2.767      0.006       0.007       0.041
idade                                         0.0049      0.001      6.399      0.000       0.003       0.006
tempo_emprego                                 0.0610      0.001     59.084      0.000       0.059       0.063
==============================================================================
Omnibus:                        1.243   Durbin-Watson:                   2.026
Prob(Omnibus):                  0.537   Jarque-Bera (JB):                1.216
Skew:                           0.022   Prob(JB):                        0.545
Kurtosis:                       3.020   Cond. No.                         226.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [257]:
df['y_chap'] = np.exp(reg.fittedvalues)
df[['y_chap', 'renda']].corr().iloc[0,1]**2

0.3615608410250814

 - Inicialmente tinhamos diversas variáveis não relevantes com valores de P-Value desequilibrados para a nossa regressão, além do R² = 35.8%. Após a remoção dos dados não relevantes obtivemos melhores valores de P-Value, além do R² = 36,1% mesmo com menor número de variáveis conseguindo manter um valor maior que o inicial e com variáveis relevantes.
 - At first, we had several non-relevant variables with imbalanced P-Value for our regression, alongside an R² = 35.8%. After removing the irrelevant data, we obtained improved P-Value values, achieving an R² = 36.1% even with a smaller number of variables. This allowed us to maintain a higher value than the initial one, while having relevant variables.